# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,375.574477,-0.595480,1234,374.978997,376.169956,749.957994,752.339913
6,86.000177,0.887992,1234,86.888169,85.112184,173.776339,170.224369
7,351.074996,-0.405325,1234,350.669671,351.480321,701.339342,702.960643
8,835.912761,-0.014551,1234,835.898209,835.927312,1671.796419,1671.854624
9,877.972502,-0.698420,1234,877.274082,878.670922,1754.548165,1757.341843
10,831.509928,0.892653,1234,832.402580,830.617275,1664.805161,1661.234550
11,365.130950,-0.087207,1234,365.043744,365.218157,730.087488,730.436314
12,464.291829,-0.408253,1234,463.883576,464.700083,927.767153,929.400165
13,471.213601,0.520539,1234,471.734140,470.693062,943.468281,941.386125
14,808.891195,-0.744540,1234,808.146654,809.635735,1616.293308,1619.271470


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-374.978997,376.169956
6,-86.888169,85.112184
7,-350.669671,351.480321
8,-835.898209,835.927312
9,-877.274082,878.670922
10,-832.402580,830.617275
11,-365.043744,365.218157
12,-463.883576,464.700083
13,-471.734140,470.693062
14,-808.146654,809.635735


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-374.978997,376.169956
6,-86.888169,85.112184
7,-350.669671,351.480321
8,-835.898209,835.927312
9,-877.274082,878.670922
10,-832.402580,830.617275
11,-365.043744,365.218157
12,-463.883576,464.700083
13,-471.734140,470.693062
14,-808.146654,809.635735


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-374.97899719,  376.16995634],
       [ -86.88816931,   85.11218445],
       [-350.66967111,  351.48032145],
       [-835.89820932,  835.92731185],
       [-877.27408244,  878.67092171],
       [-832.40258033,  830.61727512],
       [-365.04374388,  365.21815705],
       [-463.88357629,  464.70008262],
       [-471.73414026,  470.69306227],
       [-808.14665421,  809.63573503]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,747.957994,-752.339913
6,171.776339,-170.224369
7,699.339342,-702.960643
8,1669.796419,-1671.854624
9,1752.548165,-1757.341843
10,1662.805161,-1661.234550
11,728.087488,-730.436314
12,925.767153,-929.400165
13,941.468281,-941.386125
14,1614.293308,-1619.271470


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
